In [ ]:
import sys
sys.path.append("/home/iwawiwi/research/22/lipreading-lightning")
print(sys.path)

Import model class

In [ ]:
import torch
from src.models.components.lrw_video_model import VideoModel

Instantiate model then load from checkpoint

In [ ]:
# define hyperparam for the model
hparams = {
	"se":False, 
    "border":False, 
    "n_classes":500
}
# Refer to this model: "/home/iwawiwi/research/22/lipreading-lightning/data/checkpoint/lrw-cosine-lr-acc-0.85080.pt"

In [ ]:
checkpoint = torch.load("/home/iwawiwi/research/22/lipreading-lightning/data/checkpoint/lrw-cosine-lr-acc-0.85080.pt")

In [ ]:
print(checkpoint["video_model"].keys())

In [ ]:
model = VideoModel(hparams)
model.load_state_dict(checkpoint["video_model"])

In [ ]:
model.eval()
print(model)

Load test dataset to evaluate model

In [ ]:
from src.datamodules.components.lrw_dataset import LRWDataset
# import dataloader
from torch.utils.data import DataLoader

In [ ]:
testset = LRWDataset("/home/iwawiwi/research/22/lipreading-lightning/data/lrw_cropped", "test")

In [ ]:
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=4)

Evaluate model

In [ ]:
# evaluate and calculate test accuracy
with torch.no_grad():
    correct = 0
    total = 0
    for data in testloader:
        images, labels = data["video"], data["label"].long()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print("Accuracy of the model on the test images: %d %%" % (100 * correct / total))

Try to check prediction for one batch of data

In [ ]:
with torch.no_grad():
    batch = next(iter(testloader))
    videos, labels = batch["video"], batch["label"].long()
    outputs = model(videos)

print(outputs.shape)

In [ ]:
sample = outputs[0]
print(sample)

Apply softmax to output of model

In [ ]:
# apply softmax on sample
sample_softmax = torch.softmax(sample, dim=0)
print(sample_softmax)

In [ ]:
print(torch.sum(sample_softmax))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# plot sample_softmax signal
plt.plot(sample_softmax.numpy())